In [9]:
import datasets
import numpy as np

In [5]:
hf_data = datasets.load_dataset('allenai/tulu-v2-sft-mixture')

In [6]:
hf_data

DatasetDict({
    train: Dataset({
        features: ['dataset', 'id', 'messages'],
        num_rows: 326154
    })
})

In [10]:
np.unique(hf_data['train']['dataset'])

array(['code_alpaca', 'cot', 'flan_v2', 'gpt4_alpaca', 'hard_coded',
       'lima', 'oasst1', 'open_orca', 'science.evidence_inference',
       'science.qasper_truncated_4000', 'science.scierc_ner',
       'science.scierc_relation', 'science.scifact_json',
       'science.scitldr_aic', 'sharegpt', 'wizardlm'], dtype='<U29')

In [11]:
lima = hf_data.filter(lambda example: example["dataset"] == 'lima')

Filter:   0%|          | 0/326154 [00:00<?, ? examples/s]

In [21]:
from huggingface_hub import notebook_login


In [22]:
notebook_login()


In [25]:
lima.push_to_hub("VityaVitalich/LIMA")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/VityaVitalich/LIMA/commit/170acea28362e97f04d9a393a58fe992b94e9d1e', commit_message='Upload dataset', commit_description='', oid='170acea28362e97f04d9a393a58fe992b94e9d1e', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
loaded_lima = datasets.load_dataset("VityaVitalich/LIMA")

Generating train split:   0%|          | 0/1018 [00:00<?, ? examples/s]

In [27]:
loaded_lima['train']

Dataset({
    features: ['dataset', 'id', 'messages'],
    num_rows: 1018
})

In [29]:
loaded_lima['train'][1]

{'dataset': 'lima',
 'id': 'lima_1',
 'messages': [{'role': 'user',
   'content': 'In our computer systems lecture we were introduced to the MIPS processor. It was (re)developed over the course of the term and has in fact been quite easy to understand. It uses a RISC design, that is its elementary commands are regularly encoded and there are only few of them in order to keep the wires simple.\nIt was mentioned that CISC follows a different philosophy. I looked briefly at the x86 instruction set and was shocked. I can not image how anyone would want to build a processor that uses so complex a command set!\nSo I figure there have to be good arguments why large portions of the processor market use CISC architectures. What are they? '},
  {'role': 'assistant',
   'content': "There is a general historical trend.\nIn the olden days, memories were small, and so programs were perforce small. Also, compilers were not very smart, and many programs were written in assembler, so it was considered 